# Tarea Práctica Machine Learning

## Bibliioteca caret para R
**caret** es un acronimo para **C**lassification **A**nd **RE**gression **T**raining (Entrenamiento de Regresión y Clasificación).

- Caret nos permite acelerar nuestro trabajo proveyendo una interfaz común a cientos de algoritmos de Machine Learning: 
    - Este paquete nos permite utilizar más de 200 algoritmos de Machine Learning distintos.
    - Funciones para dividir los datos y muestrear los datos (Data splitting/sampling). 
    - Seleccionar características (Feature selection).
    - Tuneo de modelos.

Documentación de caret: http://topepo.github.io/caret/index.html

## Biblioteca e1071
**e1071** implementa varios algoritmos de Mahchine Learning que se pueden acceder con caret.

In [22]:
install.packages("dplyr")
install.packages("tidyverse")
install.packages("DataExplorer")
library(dplyr)
library(lattice)
library(ggplot2)
library(e1071)
library(caret)
library(DataExplorer)



The downloaded binary packages are in
	/var/folders/ly/95h1nybs1bqbfq40d29xnmdc0000gn/T//RtmpS1l1Tb/downloaded_packages

The downloaded binary packages are in
	/var/folders/ly/95h1nybs1bqbfq40d29xnmdc0000gn/T//RtmpS1l1Tb/downloaded_packages

The downloaded binary packages are in
	/var/folders/ly/95h1nybs1bqbfq40d29xnmdc0000gn/T//RtmpS1l1Tb/downloaded_packages


## Dataset **FIFA 2019**
Para este trabajo vamos a utilizar el un conjunto de datos de jugadores del juego FIFA 2019. https://www.kaggle.com/karangadiya/fifa19

Este dataset contiene informacion de los distintos jugadores del video juego FIFA 2019. El mismo contiene estadísticas de los distintos jugadores de fútbol que están en el juego.

En esta tarea vamos a intentar predecir el valor de los jugadores a partir de las distintas estadísticas.


In [23]:
# Defino url del archivo csv
fifa_url <- 'datasets/fifa19.csv'

# cargo el dataset en un dataframe
fifa <- read.table(fifa_url, sep = ",", header = TRUE, quote = "\"", encoding="UTF-8")

Observamos los datos antes de hacerles cualquier preprocesamiento

In [24]:
summary(fifa)

       X               ID                   Name            Age       
 Min.   :    0   Min.   :    16   J. Rodríguez:   11   Min.   :16.00  
 1st Qu.: 4552   1st Qu.:200316   Paulinho    :    8   1st Qu.:21.00  
 Median : 9103   Median :221759   J. Williams :    7   Median :25.00  
 Mean   : 9103   Mean   :214298   R. Williams :    7   Mean   :25.12  
 3rd Qu.:13654   3rd Qu.:236530   Felipe      :    6   3rd Qu.:28.00  
 Max.   :18206   Max.   :246620   J. Gómez    :    6   Max.   :45.00  
                                  (Other)     :18162                  
                                            Photo          Nationality   
 https://cdn.sofifa.org/players/4/19/100803.png:    1   England  : 1662  
 https://cdn.sofifa.org/players/4/19/100899.png:    1   Germany  : 1198  
 https://cdn.sofifa.org/players/4/19/101317.png:    1   Spain    : 1072  
 https://cdn.sofifa.org/players/4/19/101473.png:    1   Argentina:  937  
 https://cdn.sofifa.org/players/4/19/101488.png:    1   France

## Primero vamos a hacer una mini limpieza de los datos

Descartamos columnas que no son útiles para el aprendizaje: IDs y URLs

In [25]:
fifa <- select(fifa,-c('X', 'ID', 'Flag', 'Photo', 'Club.Logo'))

Convertimos a numéricos los valores sacandoles el símbolo de euros y los K y M al final, de miles y millones respectivamente.

In [26]:
value_wage <- function(df_value) {
    value <- substr(df_value[[1]], start=2, stop=nchar(df_value[[1]])-1)
    end <- substr(df_value[[1]], start=nchar(df_value[[1]]), stop=nchar(df_value[[1]]))
    if (identical(end, "M")) {
        value <- as.numeric(value) * 1000000 
    } else if (identical(end, 'K')) {
        value <- as.numeric(value) * 1000
    } else {
        value <- 0
    }
    return(value)
}

fifa['Value'] <- apply(select(fifa, 'Value'), 1, value_wage)
fifa['Wage'] <- apply(select(fifa, 'Wage'), 1, value_wage)
fifa['Release.Clause'] <- apply(select(fifa, 'Release.Clause'), 1, value_wage)

Descartamos algunas columnas que no vamos a utilizar

In [27]:
fifa <- select(fifa, -c('LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 
                        'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
                        'LCB', 'CB', 'RCB', 'RB'))
summary(fifa)

           Name            Age           Nationality       Overall     
 J. Rodríguez:   11   Min.   :16.00   England  : 1662   Min.   :46.00  
 Paulinho    :    8   1st Qu.:21.00   Germany  : 1198   1st Qu.:62.00  
 J. Williams :    7   Median :25.00   Spain    : 1072   Median :66.00  
 R. Williams :    7   Mean   :25.12   Argentina:  937   Mean   :66.24  
 Felipe      :    6   3rd Qu.:28.00   France   :  914   3rd Qu.:71.00  
 J. Gómez    :    6   Max.   :45.00   Brazil   :  827   Max.   :94.00  
 (Other)     :18162                   (Other)  :11597                  
   Potential                    Club           Value                Wage       
 Min.   :48.00                    :  241   Min.   :        0   Min.   :     0  
 1st Qu.:67.00   Arsenal          :   33   1st Qu.:   300000   1st Qu.:  1000  
 Median :71.00   AS Monaco        :   33   Median :   675000   Median :  3000  
 Mean   :71.31   Atlético Madrid  :   33   Mean   :  2410696   Mean   :  9731  
 3rd Qu.:75.00   Borussi

## Ahora vamos a trabajar un poco en las features

En el dataset claramente tenemos features que no son numéricas, este tipo de features es necesario transformarlas para llevarlas a algo numérico, para esto recurrimos a one_hot_encoding que nos permite llevar a numérico features no numéricas, esto se hace creando una columna nueva por cada uno de los valores posibles que puede tomar la feature.

Pare ello recurrimos a la función dummyVars

In [28]:
# aplicamos one hot enconding a Preferred.Foot es decir si es diestro o zurdo
dmy <- dummyVars(" ~ .", data = fifa['Preferred.Foot'])
dmy <- data.frame(predict(dmy, newdata = fifa))
# agregamos las columnas nuevas al dataframe
fifa[colnames(dmy)] <- dmy
# descartamos la columna vieja
fifa <- select(fifa,-c('Preferred.Foot'))

Antes de aplicar el procedimiento anterior a las distintas variables categóricas hay que tener en cuenta que si la cantidad de valores posibles para una columna es muy alta, puede darnos problemas (de memoria y dificultad para que el algoritmo aprenda), por lo tanto hay que crear agrupaciones o descartar la columna.

Por ejemplo agrupamos la posición del jugador en una más sencilla

In [29]:
simple_position <- function(df) {
    value <- df['Position'][[1]]
    if (identical(value, 'GK')) {
        return('GK')
    } else if (identical(value, 'RB') | identical(value, 'LB') |
              identical(value, 'CB') | identical(value, 'LCB') |
              identical(value, 'RCB') | identical(value, 'RWB') | 
                                          identical(value, 'LWB')) {
        return('DF')
    } else if (identical(value, 'LDM') | identical(value, 'CDM') | identical(value, 'RDM')) {
        return('DM')
    } else if (identical(value, 'LM') | identical(value, 'LCM') | identical(value, 'CM') | 
               identical(value, 'RCM') | identical(value, 'RM')) {
        return('MF')
    } else if (identical(value, 'LAM') | identical(value, 'CAM') | identical(value, 'RAM') | 
               identical(value, 'LW') | identical(value, 'RW')) {
        return('AM')
    } else if (identical(value, 'RS') | identical(value, 'ST') | identical(value, 'LS') | 
               identical(value, 'CF') | identical(value, 'LF') | identical(value, 'RF')) {
        return('ST')
    } else {
        return(value)
    }
}

fifa['Position'] <- apply(fifa, 1, simple_position)

### NOTA: Antes de seguir adelante realizar las limpiezas de datos que crea necesarias, eliminar nulos por ejemplo o descartar columnas con muchos 0's o nulos, o que crea que no aportan

In [30]:
# Se remplazan los datos vacíos por NA
fifaClean <- mutate_all(fifa, funs(na_if(.,"")))
profile_missing(fifaClean)

feature,num_missing,pct_missing
<fct>,<int>,<dbl>
Name,0,0.000000000
Age,0,0.000000000
Nationality,0,0.000000000
Overall,0,0.000000000
Potential,0,0.000000000
Club,241,0.013236667
Value,0,0.000000000
Wage,0,0.000000000
Special,0,0.000000000


In [ ]:
#Cantidad de valores $0
summarise_all(fifaClean, funs(sum(is.na(.))))

In [ ]:
#Limpiamos valores en 0
fifaNotNull <- na.omit(fifaClean)
summarise_all(fifaNotNull, funs(sum(is.na(.))))

## Dividimos los datos según las diferentes posiciones ##

In [53]:
#Dataset para Goleros
fifaGK <- fifaClean %>% filter(Position == "GK")%>% 
                   select(Name, Age, Nationality, Overall, Potential,GKDiving,
                              GKHandling,GKKicking,GKPositioning,GKReflexes) 
head(fifaGK, 20)

Name,Age,Nationality,Overall,Potential,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
<fct>,<int>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
De Gea,27,Spain,91,93,90,85,87,88,94
J. Oblak,25,Slovenia,90,93,86,92,78,88,89
M. ter Stegen,26,Germany,89,92,87,85,88,85,90
T. Courtois,26,Belgium,89,90,85,91,72,86,88
M. Neuer,32,Germany,89,89,90,86,91,87,87
H. Lloris,31,France,88,88,88,84,68,83,92
S. Handanovič,33,Slovenia,88,88,87,86,69,89,89
G. Buffon,40,Italy,88,88,88,87,74,90,83
K. Navas,31,Costa Rica,87,87,90,81,75,82,90


[1] FALSE FALSE FALSE FALSE FALSE FALSE

In [57]:
#Dataset para Defensa 
fifaDF <- fifaClean %>% filter(Position == "DF")
summary(fifaDF, 20)
#%>% select(Name, Age, Nationality, Overall, Potential, Jumping, SlidingTackle, Stamina, Strength, Interceptions, StandingTackle) 

           Name           Age                     Nationality      Overall    
 Felipe      :   5   Min.   :16.00   England            : 539   Min.   :47.0  
 M. Herrera  :   4   1st Qu.:22.00   Germany            : 393   1st Qu.:62.0  
 A. Taylor   :   3   Median :25.00   Spain              : 347   Median :66.0  
 D. Schmidt  :   3   Mean   :25.47   France             : 319   Mean   :66.4  
 Danilo      :   3   3rd Qu.:29.00   Argentina          : 286   3rd Qu.:71.0  
 J. Clarke   :   3   Max.   :41.00   Brazil             : 282   Max.   :91.0  
 J. Gómez    :   3                   Italy              : 229                 
 J. Murillo  :   3                   Colombia           : 194                 
 J. Williams :   3                   Netherlands        : 154                 
 L. Rodríguez:   3                   China PR           : 141                 
 M. Bauer    :   3                   Japan              : 138                 
 M. Martínez :   3                   Sweden         

In [ ]:
#Un algoritmo por posición
#Armar una regresión que tome en cuenta las caracteristicas más importantes


In [58]:
#Dataset para DM
fifaDM <- fifaClean %>% filter(Position == "DM")
summary(fifaDM, 20)

             Name           Age                     Nationality 
 J. Torres     :   3   Min.   :16.00   Germany            :126  
 A. Al Khaibari:   2   1st Qu.:22.00   France             : 99  
 A. Majrashi   :   2   Median :26.00   Brazil             : 90  
 Fernando      :   2   Mean   :25.82   Spain              : 87  
 J. Hernández  :   2   3rd Qu.:29.00   Argentina          : 81  
 J. Mosquera   :   2   Max.   :39.00   Colombia           : 77  
 J. Williams   :   2                   England            : 76  
 L. Kelly      :   2                   Poland             : 41  
 A. Abrashi    :   1                   Saudi Arabia       : 40  
 A. Ademi      :   1                   Turkey             : 38  
 A. Aguilar    :   1                   Chile              : 33  
 A. Al Dawsari :   1                   China PR           : 33  
 A. Al Hassan  :   1                   Republic of Ireland: 32  
 A. Al Jebreen :   1                   United States      : 31  
 A. Al Sharid  :   1     

In [59]:
#Dataset para MF
fifaMF <- fifaClean %>% filter(Position == "MF")
summary(fifaMF, 20)

            Name           Age                     Nationality  
 J. Jones     :   4   Min.   :16.00   England            : 478  
 M. García    :   3   1st Qu.:21.00   Spain              : 294  
 A. Bangoura  :   2   Median :24.00   Germany            : 277  
 A. Campbell  :   2   Mean   :24.44   Argentina          : 215  
 A. Castro    :   2   3rd Qu.:27.00   France             : 186  
 A. Ivan      :   2   Max.   :39.00   Italy              : 170  
 Allan        :   2                   Colombia           : 145  
 Álvaro García:   2                   Japan              : 135  
 Carlitos     :   2                   Sweden             : 108  
 D. Gómez     :   2                   China PR           : 107  
 D. Johnson   :   2                   Netherlands        : 103  
 D. Moreno    :   2                   Brazil             : 102  
 D. Rodríguez :   2                   Norway             : 102  
 D. Williams  :   2                   Denmark            :  99  
 D. Wright    :   2      

In [60]:
#Dataset para AM
fifaAM <- fifaClean %>% filter(Position == "AM")
summary(fifaAM, 20)

            Name           Age                Nationality     Overall     
 A. López     :   2   Min.   :16.00   Brazil        :144   Min.   :47.00  
 A. Miranchuk :   2   1st Qu.:21.00   England       :109   1st Qu.:63.00  
 A. Traoré    :   2   Median :24.00   Argentina     :102   Median :67.00  
 C. Villanueva:   2   Mean   :24.22   France        : 87   Mean   :66.97  
 Camacho      :   2   3rd Qu.:27.00   Germany       : 87   3rd Qu.:71.00  
 G. Fernández :   2   Max.   :41.00   Spain         : 85   Max.   :92.00  
 J. Hofmann   :   2                   Italy         : 76                  
 J. Rodríguez :   2                   Netherlands   : 67                  
 J. Vela      :   2                   Colombia      : 60                  
 João Teixeira:   2                   Chile         : 58                  
 K. Nakamura  :   2                   Portugal      : 44                  
 M. Çelik     :   2                   Japan         : 42                  
 M. Rodriguez :   2      

In [61]:
#Dataset para ST
fifaST <- fifaClean %>% filter(Position == "ST")
summary(fifaST, 20)

           Name           Age                     Nationality     Overall     
 A. Rodríguez:   3   Min.   :16.00   England            :281   Min.   :47.00  
 J. Martínez :   3   1st Qu.:21.00   Argentina          :155   1st Qu.:62.00  
 M. Gómez    :   3   Median :25.00   Germany            :148   Median :66.00  
 Paulinho    :   3   Mean   :24.98   Spain              :142   Mean   :66.29  
 A. Fletcher :   2   3rd Qu.:28.00   Brazil             :139   3rd Qu.:71.00  
 A. Hernández:   2   Max.   :39.00   France             :119   Max.   :94.00  
 A. Ramos    :   2                   Italy              :111                  
 Álvaro      :   2                   Colombia           : 88                  
 B. Fernández:   2                   Japan              : 75                  
 C. González :   2                   Sweden             : 63                  
 D. Williams :   2                   Chile              : 55                  
 G. Simeone  :   2                   Denmark        

### Aplicar los procedimientos anteriores a las columnas que crea necesarias

## Separar la columna value e intentar predecirla con una regresión lineal

## Utilizar validación cruzada y estimar el error del clasificador